In [15]:
import os
from dotenv import load_dotenv
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import pandas as pd
import uuid
from typing import Dict, List
from sqlalchemy import make_url
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from tqdm import tqdm
from llama_index.core.node_parser import SimpleNodeParser

In [16]:
# load environment variables
load_dotenv()

# may need to change this to a different name
openai_api_key = os.getenv("OPENAI_API_KEY")
connection_string = os.getenv("DB_CONNECTION") 

url = make_url(connection_string)

In [17]:
def generate_unique_id(name: str, school: str) -> str:
    """Generate a unique ID based on name and school"""
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{name}-{school}"))

def create_embedding_text(row: pd.Series) -> str:
    """Create a concatenated text string for embedding"""
    # Start with required fields using column names
    text_parts = [
        f"Major: {row['Major(s) and Minor(s)']}", 
        f"Degree: {row['Degree(s) Earned']}", 
        f"Job Title: {row['Current Job Title']}", 
        f"Industry: {row['Industry']}"
    ]
    
    # Add comments without column name if they exist
    if not pd.isna(row['Comment']):
        text_parts.append(str(row['Comment']))
        
    return "; ".join(text_parts)

def prepare_metadata(row: pd.Series) -> Dict:
    """Create metadata dictionary for each record"""
    metadata = {
        "grad_year": int(row["Graduation Year"]),
        "degree": row["Degree(s) Earned"],
        "major": row["Major(s) and Minor(s)"],
        "job_title": row["Current Job Title"],
        "industry": row["Industry"],
        "school": row["School"],
        "comments": None if pd.isna(row['Comment']) else row['Comment'],
        "name": row["Name/Identifier"],
        "source_identifier": f"{row['Name/Identifier']}-{row['School']}"
    }
    return metadata

def process_alumni_data(df: pd.DataFrame) -> List[Document]:
    """Process alumni data and return LlamaIndex documents"""
    documents = []
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        original_id = generate_unique_id(row["Name/Identifier"], row["School"])
        
        metadata = prepare_metadata(row)
        metadata["original_id"] = original_id
        
        doc = Document(
            text=create_embedding_text(row),
            metadata=metadata
        )
        documents.append(doc)
    
    return documents


In [38]:
df_raw_1 = pd.read_excel("..\\data\\raw_alumini_data_billy_1.xlsx")
df_raw_2 = pd.read_excel("..\\data\\raw_alumini_data_billy_2.xlsx")
df_raw_3 = pd.read_excel("..\\data\\raw_alumini_data_sierra_1.xlsx")


In [39]:
# check the column names and types for each dataframe
print(df_raw_1.info())
print(df_raw_2.info())
print(df_raw_3.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name/Identifier        51 non-null     object 
 1   Graduation Year        51 non-null     int64  
 2   Degree(s) Earned       51 non-null     object 
 3   Major(s) and Minor(s)  51 non-null     object 
 4   Current Job Title      51 non-null     object 
 5   Industry               51 non-null     object 
 6   School                 51 non-null     object 
 7   Comment                0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 3.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name/Identifier        50 non-null     object 
 1   Graduation Year        50 non-null     int64  
 

In [40]:
df_combined = pd.concat([df_raw_1, df_raw_2, df_raw_3])

# check unique values for each column
print(df_combined["School"].unique())
print(df_combined["Major(s) and Minor(s)"].unique())
print(df_combined["Degree(s) Earned"].unique())


['CGU' 'Purdue University' 'Northwestern University' 'UCLA' 'UC Davis'
 'Loyola Marymount University' 'University of Illinois Urbana-Champaign'
 'University of Chicago' 'Carnegie Mellon University'
 'Columbia University' 'Caltech' 'University of Michigan' 'Pomona College'
 'Yele University' 'Duke University' 'Northeastern University\xa0'
 'James Cook University' 'San Diego State University'
 'Scripps Institution of Oceanography' 'CalPoly San Luis Obispo'
 'Wheaton College Massachusetts' 'Cal State Long Beach'
 'Portland State University' 'University of Maine' 'Not CGU']
['Financial Engineering' 'Information Systems and Technology' 'Mathmatics'
 'Economics and Financial Engineering' 'Computational Mathmatics'
 'Business Administration' 'Economics'
 'Applied Biostatistics and Epidemiology' 'Human Resources Design'
 'Fine Arts' 'Public Health' 'Education' 'Human Resources'
 'Data Evaluation and Cognitive Science'
 'English Language and Literature/Lertters' 'Management'
 'Applied Social Ps

In [41]:
# make the school column 'CGU' or 'Not CGU'
df_combined["School"] = df_combined["School"].apply(lambda x: 'CGU' if x == 'CGU' else 'Not CGU')

# fix the degree column to 'Doctorate' or 'PhD'
df_combined["Degree(s) Earned"] = df_combined["Degree(s) Earned"].apply(lambda x: 'Doctorate' if x == 'PhD' else x)


In [42]:
df_combined["School"].value_counts()

School
Not CGU    112
CGU         51
Name: count, dtype: int64

In [43]:
df_combined["Degree(s) Earned"].value_counts()



Degree(s) Earned
M.S          78
Doctorate    41
B.S          17
MBA           7
MA            7
BA            5
MPH           3
MFA           1
EdD           1
LL.M.         1
JD            1
A.S           1
Name: count, dtype: int64

In [44]:
df_combined.shape

(163, 8)

In [45]:
# check for missing values
df_combined.isnull().sum()


Name/Identifier            0
Graduation Year            0
Degree(s) Earned           0
Major(s) and Minor(s)      0
Current Job Title          0
Industry                   0
School                     0
Comment                  163
dtype: int64

In [46]:
# Process the data
documents_raw = process_alumni_data(df_combined)

100%|██████████| 163/163 [00:00<00:00, 15387.61it/s]


In [47]:
parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(documents_raw)
print(f'{len(nodes)} nodes created')

163 nodes created


In [48]:

# Create embedding model
embedding_model = OpenAIEmbedding(model="text-embedding-3-large")

# Set up database connection and vector store
url = make_url(connection_string)
vector_store = PGVectorStore.from_params(
    database='ai_advising_db',
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="alumni_records",
    embed_dim=3072,  # openai embedding dimension
    hybrid_search=True,  # Enable hybrid search
    text_search_config="english",
)

# Create storage context and index
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [49]:
documents_raw[0]

Document(id_='45855fbe-ae1b-4392-83ba-249ba7dbf696', embedding=None, metadata={'grad_year': 2012, 'degree': 'M.S', 'major': 'Financial Engineering', 'job_title': 'Engineering Leader', 'industry': 'Entertainment Providers', 'school': 'CGU', 'comments': None, 'name': 'Yunzhi Z.', 'source_identifier': 'Yunzhi Z.-CGU', 'original_id': 'bbec7767-7721-5682-99e6-463bfdaf93b4'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Major: Financial Engineering; Degree: M.S; Job Title: Engineering Leader; Industry: Entertainment Providers', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [50]:
nodes[0]

TextNode(id_='0a0a0d0c-94c6-47f6-a075-fb8308321fcb', embedding=None, metadata={'grad_year': 2012, 'degree': 'M.S', 'major': 'Financial Engineering', 'job_title': 'Engineering Leader', 'industry': 'Entertainment Providers', 'school': 'CGU', 'comments': None, 'name': 'Yunzhi Z.', 'source_identifier': 'Yunzhi Z.-CGU', 'original_id': 'bbec7767-7721-5682-99e6-463bfdaf93b4'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='45855fbe-ae1b-4392-83ba-249ba7dbf696', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'grad_year': 2012, 'degree': 'M.S', 'major': 'Financial Engineering', 'job_title': 'Engineering Leader', 'industry': 'Entertainment Providers', 'school': 'CGU', 'comments': None, 'name': 'Yunzhi Z.', 'source_identifier': 'Yunzhi Z.-CGU', 'original_id': 'bbec7767-7721-5682-99e6-463bfdaf93b4'}, hash='162b5e64f1babd24d9dbb85d5c6c0f066b16be1ab9590518fd26ee3ef1ba5217')}, text='Major: Financial Engineering;

In [51]:
# Create and store index
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    embed_model=embedding_model,
    show_progress=True
)

Generating embeddings: 100%|██████████| 163/163 [00:02<00:00, 62.84it/s]
